# Pasos previos

En el módulo anterior, habíamos aplicado nuestra [transformación de datos](https://github.com/IrvingC48/BeduFase2-Proyecto_python/blob/main/Procesos/6_Transformacion_de_datos.md) sin eliminar los datos atípicos.  
Por lo que antes de antes de verificar los estimados de locación y variabilidad, debemos ajustar nuestro dataset previo al cálculo del promedio diario y el índice de calidad del aire.

Para nuestra transformación de datos, utilizaremos las siguientes librerias:

In [1]:
import pandas as pd
import numpy as np

## Transformación de $PM_{10}$

- Cargando dataset a un DataFrame de pandas
- Posteriormente normalizando la columna `Fecha` a tipo `datetime`

In [2]:
#Cargando archivos
df_PM10 = pd.read_csv('../Datasets/PM10/PM10_depurado.csv')

# Normalizando fechas
df_PM10['FECHA'] = pd.to_datetime(df_PM10['FECHA'], unit='ns')
df_PM10.head()

,Unnamed: 0,FECHA,HORA,ACO,AJM,ATI,BJU,CAM,CUA,CUT,...,IZT,MER,PED,SAG,SFE,TAH,TLA,UIZ,VIF,XAL
0,0,2019-01-01,1,201.0,39.0,105.0,84.0,120.0,113.0,149.0,...,132.0,113.0,NaN,149.0,50.0,191.0,78.0,129.0,280.0,155.0
1,1,2019-01-01,2,218.0,27.0,128.0,115.0,135.0,158.0,211.0,...,202.0,141.0,NaN,149.0,54.0,225.0,73.0,125.0,257.0,236.0
2,2,2019-01-01,3,240.0,20.0,93.0,134.0,177.0,192.0,214.0,...,198.0,118.0,NaN,152.0,65.0,185.0,80.0,132.0,509.0,337.0
3,3,2019-01-01,4,231.0,10.0,86.0,124.0,204.0,143.0,178.0,...,192.0,154.0,NaN,217.0,33.0,172.0,84.0,136.0,611.0,271.0
4,4,2019-01-01,5,216.0,7.0,68.0,143.0,162.0,123.0,234.0,...,178.0,227.0,NaN,242.0,29.0,83.0,63.0,191.0,653.0,226.0


Posteriormente, se `filtran` los datos por los meses de interés, que son `marzo`, `abril` y `mayo`. Primeramente se crea el rango de fechas de interés y luego éste se utiliza para filtrar el dataset

In [3]:
date_rng = pd.date_range(start='2019/03/01', end='2019/05/31', freq='d')#Rango en 2019
date_rng2= pd.date_range(start='2020/03/01', end='2020/05/31', freq='d') #Rango en 2020
date_rng= date_rng.append(date_rng2) #unión de rangos

In [4]:
df_PM10=df_PM10[df_PM10["FECHA"].isin(date_rng)]
df_PM10.head()

,Unnamed: 0,FECHA,HORA,ACO,AJM,ATI,BJU,CAM,CUA,CUT,...,IZT,MER,PED,SAG,SFE,TAH,TLA,UIZ,VIF,XAL
48,48,2019-03-01,1,119.0,20.0,62.0,27.0,35.0,23.0,71.0,...,64.0,72.0,NaN,110.0,13.0,23.0,40.0,37.0,130.0,96.0
49,49,2019-03-01,2,90.0,34.0,48.0,29.0,27.0,20.0,53.0,...,46.0,80.0,NaN,133.0,22.0,25.0,51.0,47.0,87.0,114.0
50,50,2019-03-01,3,81.0,47.0,37.0,26.0,21.0,28.0,41.0,...,44.0,49.0,NaN,124.0,27.0,36.0,55.0,47.0,66.0,128.0
51,51,2019-03-01,4,65.0,44.0,39.0,39.0,58.0,38.0,39.0,...,51.0,38.0,NaN,70.0,38.0,17.0,25.0,47.0,56.0,118.0
52,52,2019-03-01,5,53.0,40.0,48.0,32.0,63.0,12.0,35.0,...,56.0,55.0,NaN,90.0,26.0,23.0,27.0,42.0,156.0,130.0


- Reacomodo del DataFrame, donde ahora tendremos una columna para las `estaciones` y sus correspondientes `datos de medición`
- Eliminando los datos `NaN`
- Comprobando que no queden datos `NaN` en el DataFrame

In [5]:
# Extraer nombre de columnas
cols = df_PM10.columns.tolist()

# Reacomodando dataframe
df_PM10 = pd.melt(df_PM10, id_vars=['FECHA', 'HORA'], value_vars=cols[3:], var_name='station', value_name='measurement')

# df = df[~df['measurement'].isna()] otra forma de eliminar nan's
df_PM10 = df_PM10.dropna(axis=0, how='any').reset_index(drop=True)

print("No. de columnas que contienen valores nulos ")
print(len(df_PM10.columns[df_PM10.isna().any()]))

print("No. de columnas que no contienen valores nulos")
print(len(df_PM10.columns[df_PM10.notna().all()]))

print("No total de columnas en el dataframe")
print(len(df_PM10.columns))
df_PM10.head()

No. de columnas que contienen valores nulos 
0
No. de columnas que no contienen valores nulos
4
No total de columnas en el dataframe
4


,FECHA,HORA,station,measurement
0,2019-03-01,1,ACO,119.0
1,2019-03-01,2,ACO,90.0
2,2019-03-01,3,ACO,81.0
3,2019-03-01,4,ACO,65.0
4,2019-03-01,5,ACO,53.0


Ordenando el dataframe segun la fecha y la estación

In [6]:
df_PM10 = df_PM10.sort_values(['FECHA', 'station']).reset_index(drop=True)
df_PM10

,FECHA,HORA,station,measurement
0,2019-03-01,1,ACO,119.0
1,2019-03-01,2,ACO,90.0
2,2019-03-01,3,ACO,81.0
3,2019-03-01,4,ACO,65.0
4,2019-03-01,5,ACO,53.0
...,...,...,...,...
67375,2020-05-31,20,VIF,12.0
67376,2020-05-31,21,VIF,5.0
67377,2020-05-31,22,VIF,13.0
67378,2020-05-31,23,VIF,21.0


`Agregamos` una columna para el `año` con el nombre `Year`, y otro para el `mes` con el nombre `Month`

In [7]:
df_PM10.insert(loc=1,column='Year',value=df_PM10['FECHA'].dt.year)
df_PM10.insert(loc=2,column='Month',value=df_PM10['FECHA'].dt.month)
df_PM10.head()

,FECHA,Year,Month,HORA,station,measurement
0,2019-03-01,2019,3,1,ACO,119.0
1,2019-03-01,2019,3,2,ACO,90.0
2,2019-03-01,2019,3,3,ACO,81.0
3,2019-03-01,2019,3,4,ACO,65.0
4,2019-03-01,2019,3,5,ACO,53.0


Se procede a relacionar cada estación con su respectiva zona de activación. Primeramente se leen los datos

In [8]:
# Cargando dataset de zonas
df_zonas = pd.read_csv('../Datasets/cat_estacion_depurado.csv')

# Eliminando columna 'Unnamed: 0'
df_zonas = df_zonas.drop(columns=['Unnamed: 0'])
# Remplazando los - por NaN
df_zonas = df_zonas.replace('-', np.nan)
# Eliminando lo NaN's
df_zonas = df_zonas.dropna(axis=0, how='any').reset_index(drop=True)
# Renombrando la columna
df_zonas = df_zonas.rename(columns={'cve_estac': 'station'})
df_zonas.head()

,station,nom_estac,Zona
0,ACO,Acolman,NE
1,AJU,Ajusco,SO
2,AJM,Ajusco Medio,SO
3,ATI,Atizapan,NO
4,BJU,Benito Ju�rez,CE


Uniendo los datos con las zonas correspondientes

In [9]:
df_PM10 = df_PM10.merge(df_zonas[['station', 'Zona']], on='station')
df_PM10.head()

,FECHA,Year,Month,HORA,station,measurement,Zona
0,2019-03-01,2019,3,1,ACO,119.0,NE
1,2019-03-01,2019,3,2,ACO,90.0,NE
2,2019-03-01,2019,3,3,ACO,81.0,NE
3,2019-03-01,2019,3,4,ACO,65.0,NE
4,2019-03-01,2019,3,5,ACO,53.0,NE


Guardamos nuestro dataset de está partícula para el analisis de este modulo

In [10]:
df_PM10.to_csv('../Datasets/PM10/PM10_depurado_melt.csv')

## Transformación de $PM_{2.5}$

- Cargando dataset a un DataFrame de pandas
- Posteriormente normalizando la columna `Fecha` a tipo `datetime`

In [11]:
#Cargando archivos
df_PM25 = pd.read_csv('../Datasets/PM2.5/PM25_depurado.csv')

# Normalizando fechas
df_PM25['FECHA'] = pd.to_datetime(df_PM25['FECHA'], unit='ns')
df_PM25.head()

,Unnamed: 0,FECHA,HORA,AJM,AJU,BJU,CAM,CCA,FAR,GAM,...,MON,NEZ,PED,SAC,SAG,SFE,TLA,UAX,UIZ,XAL
0,0,2019-01-01,1,19.0,35.0,62.0,90.0,66.0,NaN,NaN,...,102.0,133.0,NaN,NaN,92.0,33.0,55.0,74.0,99.0,96.0
1,1,2019-01-01,2,17.0,24.0,88.0,104.0,84.0,NaN,NaN,...,108.0,164.0,NaN,NaN,100.0,40.0,52.0,115.0,99.0,164.0
2,2,2019-01-01,3,14.0,20.0,107.0,140.0,95.0,NaN,NaN,...,125.0,206.0,NaN,NaN,104.0,52.0,59.0,150.0,109.0,249.0
3,3,2019-01-01,4,6.0,15.0,101.0,162.0,97.0,NaN,NaN,...,92.0,273.0,NaN,NaN,144.0,25.0,58.0,180.0,112.0,200.0
4,4,2019-01-01,5,4.0,8.0,121.0,133.0,88.0,NaN,NaN,...,84.0,291.0,NaN,NaN,171.0,21.0,46.0,167.0,164.0,161.0


Posteriormente, se `filtran` los datos por los meses de interés, que son `marzo`, `abril` y `mayo`. Primeramente se crea el rango de fechas de interés y luego éste se utiliza para filtrar el dataset

In [12]:
date_rng = pd.date_range(start='2019/03/01', end='2019/05/31', freq='d')#Rango en 2019
date_rng2= pd.date_range(start='2020/03/01', end='2020/05/31', freq='d') #Rango en 2020
date_rng= date_rng.append(date_rng2) #unión de rangos

In [13]:
df_PM25=df_PM25[df_PM25["FECHA"].isin(date_rng)]
df_PM25.head()

,Unnamed: 0,FECHA,HORA,AJM,AJU,BJU,CAM,CCA,FAR,GAM,...,MON,NEZ,PED,SAC,SAG,SFE,TLA,UAX,UIZ,XAL
48,48,2019-03-01,1,8.0,1.0,12.0,14.0,12.0,NaN,34.0,...,12.0,31.0,NaN,NaN,40.0,5.0,18.0,27.0,17.0,46.0
49,49,2019-03-01,2,19.0,4.0,14.0,13.0,11.0,NaN,31.0,...,7.0,35.0,NaN,NaN,44.0,12.0,33.0,27.0,26.0,51.0
50,50,2019-03-01,3,29.0,3.0,11.0,12.0,15.0,NaN,35.0,...,13.0,30.0,NaN,NaN,44.0,16.0,35.0,37.0,29.0,54.0
51,51,2019-03-01,4,28.0,6.0,22.0,17.0,18.0,NaN,40.0,...,18.0,32.0,NaN,NaN,30.0,27.0,13.0,29.0,29.0,54.0
52,52,2019-03-01,5,26.0,3.0,18.0,24.0,18.0,NaN,41.0,...,14.0,34.0,NaN,NaN,33.0,17.0,14.0,17.0,24.0,62.0


- Reacomodo del DataFrame, donde ahora tendremos una columna para las `estaciones` y sus correspondientes `datos de medición`
- Eliminando los datos `NaN`
- Comprobando que no queden datos `NaN` en el DataFrame

In [14]:
# Extraer nombre de columnas
cols = df_PM25.columns.tolist()

# Reacomodando dataframe
df_PM25 = pd.melt(df_PM25, id_vars=['FECHA', 'HORA'], value_vars=cols[3:], var_name='station', value_name='measurement')

# df = df[~df['measurement'].isna()]
df_PM25 = df_PM25.dropna(axis=0, how='any').reset_index(drop=True)

print("No. de columnas que contienen valores nulos ")
print(len(df_PM25.columns[df_PM25.isna().any()]))

print("No. de columnas que no contienen valores nulos")
print(len(df_PM25.columns[df_PM25.notna().all()]))

print("No total de columnas en el dataframe")
print(len(df_PM25.columns))
df_PM25.head()

No. de columnas que contienen valores nulos 
0
No. de columnas que no contienen valores nulos
4
No total de columnas en el dataframe
4


,FECHA,HORA,station,measurement
0,2019-03-01,1,AJM,8.0
1,2019-03-01,2,AJM,19.0
2,2019-03-01,3,AJM,29.0
3,2019-03-01,4,AJM,28.0
4,2019-03-01,5,AJM,26.0


Ordenando el dataframe segun la fecha y la estación

In [15]:
df_PM25 = df_PM25.sort_values(['FECHA', 'station']).reset_index(drop=True)
df_PM25

,FECHA,HORA,station,measurement
0,2019-03-01,1,AJM,8.0
1,2019-03-01,2,AJM,19.0
2,2019-03-01,3,AJM,29.0
3,2019-03-01,4,AJM,28.0
4,2019-03-01,5,AJM,26.0
...,...,...,...,...
62722,2020-05-31,20,UIZ,11.0
62723,2020-05-31,21,UIZ,14.0
62724,2020-05-31,22,UIZ,5.0
62725,2020-05-31,23,UIZ,10.0


`Agregamos` una columna para el `año` con el nombre `Year`, y otro para el `mes` con el nombre `Month`

In [16]:
df_PM25.insert(loc=1,column='Year',value=df_PM25['FECHA'].dt.year)
df_PM25.insert(loc=2,column='Month',value=df_PM25['FECHA'].dt.month)
df_PM25.head()

,FECHA,Year,Month,HORA,station,measurement
0,2019-03-01,2019,3,1,AJM,8.0
1,2019-03-01,2019,3,2,AJM,19.0
2,2019-03-01,2019,3,3,AJM,29.0
3,2019-03-01,2019,3,4,AJM,28.0
4,2019-03-01,2019,3,5,AJM,26.0


Se procede a relacionar cada estación con su respectiva zona de activación. Primeramente se leen los datos

In [17]:
# Cargando dataset de zonas
df_zonas = pd.read_csv('../Datasets/cat_estacion_depurado.csv')

# Eliminando columna 'Unnamed: 0'
df_zonas = df_zonas.drop(columns=['Unnamed: 0'])
# Remplazando los - por NaN
df_zonas = df_zonas.replace('-', np.nan)
# Eliminando lo NaN's
df_zonas = df_zonas.dropna(axis=0, how='any').reset_index(drop=True)
# Renombrando la columna
df_zonas = df_zonas.rename(columns={'cve_estac': 'station'})
df_zonas.head()

,station,nom_estac,Zona
0,ACO,Acolman,NE
1,AJU,Ajusco,SO
2,AJM,Ajusco Medio,SO
3,ATI,Atizapan,NO
4,BJU,Benito Ju�rez,CE


Uniendo los datos con las zonas correspondientes

In [18]:
df_PM25 = df_PM25.merge(df_zonas[['station', 'Zona']], on='station')
df_PM25.head()

,FECHA,Year,Month,HORA,station,measurement,Zona
0,2019-03-01,2019,3,1,AJM,8.0,SO
1,2019-03-01,2019,3,2,AJM,19.0,SO
2,2019-03-01,2019,3,3,AJM,29.0,SO
3,2019-03-01,2019,3,4,AJM,28.0,SO
4,2019-03-01,2019,3,5,AJM,26.0,SO


Guardamos nuestro dataset de está partícula para el analisis de este modulo

In [19]:
df_PM25.to_csv('../Datasets/PM2.5/PM25_depurado_melt.csv')